**Learning outcomes**

* Prove why we cannot design a quantum circuit that can clone arbitrary states.
* Describe the quantum teleportation protocol.

In [2]:
import numpy as np
import pennylane as qml

**Codercise I.15.1.** (a) Prepare your favourite single-qubit quantum state on the first qubit in the `state_preparation` function below. It can be anything, as long as it's a single qubit state (you can even leave the function as-is, or make it parametrized!).

Note that this quantum function will be used as a subroutine; therefore you do not need to return a measurement or create a QNode.

In [ ]:
def state_preparation(phi, theta, omega):

    # OPTIONALLY UPDATE THIS STATE PREPARATION ROUTINE

    qml.Hadamard(wires=0)
    qml.Rot(phi,theta,omega, wires = 0)

(b) Run your circuit below to see the state you will teleport. If you made a custom state preparation function, please copy it down from the cell above and use it as a replacement for the `state_preparation` function below.

In [3]:
dev = qml.device("default.qubit", wires=1)

# OPTIONALLY REPLACE THIS STATE PREPARATION ROUTINE WITH
# THE ONE FROM THE PREVIOUS EXERCISE

def state_preparation(phi, theta, omega):

    # OPTIONALLY UPDATE THIS STATE PREPARATION ROUTINE

    qml.Hadamard(wires=0)
    qml.Rot(phi,theta,omega, wires = 0)


@qml.qnode(dev)
def state_prep_only():
    state_preparation(phi = 0.1, theta=0.2, omega=0.3)
    return qml.state()

print(state_prep_only())

[0.61930934-0.13273109j 0.75978977+0.14682614j]


**Codercise I.15.2.**
Write a quantum function that prepares a shared entangled state on the second and third qubits.

Recall the teleportation circuit looks like this:
![circuit](./images/I.15.2.1.png)
This function will also be used as a subroutine, so you do not need to return any measurements.

In [ ]:
def entangle_qubits():

# ENTANGLE THE SECOND QUBIT (WIRES=1) AND THE THIRD QUBIT

